In [1]:
import os
import sys
import time
import napari
import pickle
import h5py
import importlib
import numpy as np
import pandas as pd

from napari import Viewer

from skimage import measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import white_tophat, disk

from napari.qt.threading import thread_worker
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

from tifffile import imread,imsave,TiffFile
import matplotlib.pyplot as plt
import seaborn as sn

from nd2reader import ND2Reader
import btrack

from magicgui import magicgui

sys.path.append(r'D:\imPy\libraries') 
sys.path.append(r'D:\BARC\napari_tracking_manual')

gallery_functions = importlib.import_module('gallery_functions')
my_napari = importlib.import_module('napari_display_functions')
fov_f = importlib.import_module('fovRingsLibrary')
gen = importlib.import_module('general_functions')

In [59]:
importlib.reload(gallery_functions)
importlib.reload(my_napari)
importlib.reload(fov_f)
importlib.reload(gen)

<module 'general_functions' from 'D:\\BARC\\napari_tracking_manual\\general_functions.py'>

In [3]:
myFov = '01'

myDirIm = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tiffs'

# specify tracking channel
myFile_track_im = f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_01.tif'
track_intensity = 'intensity_01_nuc_corr' # name of the column

# specify additional channels
myFile_signal_im = [f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_02.tif',
                    f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_03.tif',
                   f'20210618_RPE_p21_cycD1_DHB_H2B_series_{myFov}_ch_04.tif']

colors_list = ['red','green','magenta']
names_list = ['DHB','cyclinD','p21']

# specify columns to plot
to_plot_signals = ['DHB_ratio','intensity_03_nuc_corr','intensity_04_nuc_corr','cyc_D_over_p21'] # name of the column
to_plot_names = ['DHB_ratio [cyt/nuc]','cyclinD','p21','Cyclin D/p21 (mean)']
to_plot_colors = ['red','green','blue','black']


myDirTracks = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
myDirTracks = os.path.join(myDirTracks,myFov)
dfDir = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking'
dfDir = os.path.join(dfDir,f'{myFov}_napari')

In [4]:
modelPath = r'D:\BARC\Sonja_2021\b_track\cell_config.json'

btrack_init_path = os.path.join(dfDir,f'{myFov}_tracks_init.h5')
btrack_mod_path = os.path.join(dfDir,f'{myFov}_tracks_mod.h5')

## Read in the tracking channel

In [5]:
%%time

# read in image
myIm = imread(os.path.join(myDirIm,myFile_track_im))

Wall time: 50.6 s


## Read in signal channels

In [6]:
%%time

myIm_signal_list = []
# read in additional signal images
for myFile in myFile_signal_im:
    temp = imread(os.path.join(myDirIm,myFile))
    myIm_signal_list.append(temp)

Wall time: 2min 32s


## Read in tracks in a form of labels

In [7]:
%%time

# read in labels as tracks
myLabels =[]

fovFiles = [x for x in os.listdir(myDirTracks) if f'series_{myFov}_' in x]

for myFile in fovFiles:
    
    # read a mask
    labels = imread(os.path.join(myDirTracks,myFile))
    
    myLabels.append(labels)
    
myLabels = np.array(myLabels)

Wall time: 1min 21s


## Get latest tracking

In [8]:
# check if modified tracking exists

try:
    f = open(btrack_mod_path.replace('h5','pkl'),'rb')
    
    print('Opening corrected tracking.')

except(FileNotFoundError):
  
    f = open(btrack_init_path.replace('h5','pkl'),'rb')
    print('Opening original tracking.')

data, properties, graph = pickle.load(f)

Opening original tracking.


## Get the full data frame

In [9]:
cellDataAll=pd.read_pickle(os.path.join(dfDir,f'cellPose_btrack_regionprops_bck_{myFov}.pkl'))
cellDataAll.drop(['file','x','y'],axis=1,inplace=True)

In [10]:
# create an array for visualization of accepted points
selData=cellDataAll.loc[cellDataAll.accepted==True,:]
acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

In [11]:
cellDataAll.head()

,label,area,centroid-0,centroid-1,orientation,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,...,intensity_02_nuc_corr,intensity_02_ring_corr,background_03,intensity_03_nuc_corr,intensity_03_ring_corr,background_04,intensity_04_nuc_corr,intensity_04_ring_corr,DHB_ratio,cyc_D_over_p21
0,1.0,353.0,7.277620,2400.410765,-1.410806,25.969168,18.036581,0.0,2388.0,18.0,...,1205.090791,227.170204,748.129512,227.595700,33.141729,1934.079928,-41.558682,-31.655091,0.188509,-5.476490
1,2.0,632.0,26.928797,1711.330696,-0.934733,30.203461,26.730650,13.0,1697.0,42.0,...,322.688395,388.158446,748.129512,59.300867,29.218314,1934.079928,-27.586257,-14.976667,1.202889,-2.149653
2,3.0,511.0,29.324853,2736.291585,-1.315686,38.273719,17.379927,19.0,2717.0,39.0,...,246.690931,27.343267,748.129512,231.623912,52.668360,1934.079928,7.828095,-15.663261,0.110840,29.588795
3,4.0,555.0,49.800000,1879.099099,-1.228809,30.717216,23.097163,38.0,1865.0,62.0,...,105.312157,127.468386,748.129512,16.958776,-6.331781,1934.079928,-63.663712,-66.399399,1.210386,-0.266381
4,5.0,423.0,56.040189,286.456265,-1.239673,30.591690,17.750233,46.0,273.0,66.0,...,369.661234,66.441352,748.129512,238.402402,39.864340,1934.079928,-22.744231,-18.465174,0.179736,-10.481885


In [31]:
cellDataAll = cellDataAll.sort_values(by=['track_id','t'])

In [60]:
%%time
# generate data for the tracking layer
data,properties,graph = gen.trackData_from_df(cellDataAll)

Wall time: 49 ms


## Main viewer

In [61]:
viewer = napari.Viewer()

# tracking layers
viewer.add_image(myIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

track_layer=viewer.add_tracks(data, properties=properties, graph=graph,name='tracking')
track_layer.display_id=True

layer_inProgress = viewer.add_labels(myLabels,name='objects',opacity = 0.5)

# signal layers
for myIm_signal,myColor,myName in zip(myIm_signal_list,colors_list,names_list):
    viewer.add_image(myIm_signal,colormap=myColor,contrast_limits=(0, 4000),opacity = 1,visible=False,name = myName)

# helper layers
layer_accepted = viewer.add_points(acceptedPoints,name='accepted objects',face_color='green', opacity =0.5, ndim=3)
layer_mod = viewer.add_points([],name='modPoints',face_color='red',ndim=3)

In [347]:
def find_all_paths(graph, node, path=[]):
    
    path = path + [node]
    paths = [path]
    
    offspring_list = []
    for key, value in graph.items():   # iter on both keys and values
            if (value == [node]):
                offspring_list.append(key)
    
    for node in offspring_list:
        newpaths = find_all_paths(graph, node, path)
        for newpath in newpaths:
            paths.append(newpath)
            
    return paths

In [365]:
%%time

# development of cutting of a track

# get the position in time
myT = viewer.dims.current_step[0]

# get my label
myLabel = layer_inProgress.selected_label

# find new track number
newTrack = gen.newTrack_number(cellDataAll.track_id)

#####################################################################
# change labels layer
#####################################################################
# look up stop frame
stopFrame = int(np.max(data[data[:,0]==myLabel,1]))

orgImShape = myLabels.shape


for myInd in np.where(data[:,0]==myLabel)[0]:

    x = int(data[myInd,2])
    y = int(data[myInd,3])
    myFrame = int(data[myInd,1])  

    # calculate cut parameters
    row_start,row_stop,column_start,column_stop,row_in_start,row_in_stop,column_in_start,column_in_stop = gallery_functions.calculate_cut(100,orgImShape,x,y) 


    temp = myLabels[myFrame,row_start:row_stop,column_start:column_stop]
    temp[temp == myLabel] = newTrack
    
    myLabels[myFrame,row_start:row_stop,column_start:column_stop] = temp
    
layer_inProgress.data = myLabels

########################################################
# modify data frame
########################################################
# find info about the cut track
myLabel_generation = list(cellDataAll.loc[cellDataAll.track_id==myLabel,'generation'].drop_duplicates())[0]

# find kids
kids_list = []
for key, value in graph.items():   # iter on both keys and values
        if (value == [myLabel]):
            kids_list.append(key)

# find all family members
all_paths = find_all_paths(graph, myLabel)
family_members = [item for sublist in all_paths for item in sublist]

for myDescendant in family_members:
    
    # find which rows need to be changed
    changeIndex = (cellDataAll.t>=myT) & (cellDataAll.track_id==myDescendant)
    
    cellDataAll.loc[changeIndex,'root'] = newTrack
    cellDataAll.loc[changeIndex,'generation'] = cellDataAll.loc[changeIndex,'generation'] - myLabel_generation
    
    if(myDescendant == myLabel):
    
        cellDataAll.loc[changeIndex,'track_id'] = newTrack
        cellDataAll.loc[changeIndex,'parent'] = newTrack
          
    elif (myDescendant in kids_list): #2nd generation
        
        cellDataAll.loc[changeIndex,'parent'] = newTrack
'''     
#####################################################################
# change tracking layer
#####################################################################

# get current status of a tracking layer
data = viewer.layers['tracking'].data
properties = viewer.layers['tracking'].properties

# modify the data for the layer
data, properties = gen.mod_trackLayer(data,properties,cellDataAll,myT,myLabel)

# change tracks layer
viewer.layers['tracking'].data = data
viewer.layers['tracking'].properties = properties

'''   
########################################################
# change viewer status
########################################################
viewer.status = f'Track {myLabel} was cut at frame {myT}.' 

Wall time: 297 ms


In [13]:
%%time

# get the position in time
myT = viewer.dims.current_step[0]

# get my label
myLabel = layer_inProgress.selected_label

# find new track number
newTrack = gen.newTrack_number(cellDataAll.track_id)

# find position of this cell in the tracking data structure
changeIndex = ((data[:,1]>=myT) & (data[:,0]==myLabel))

data[changeIndex,0] = newTrack

Wall time: 36 ms


In [30]:
%%time
track_layer.data = data
track_layer.properties = properties

Wall time: 605 ms


In [29]:
%%time

# get data and properties from the data frame into a tracking layer forma

cellDataAll = cellDataAll.sort_values(by=['track_id','t'])
data_df = np.array(cellDataAll.loc[:,['track_id','t','centroid-0','centroid-1']])
selVector = (data_df[:,0]==data_df[:,0])
data_df = data_df[selVector,:] # drop objects that don't belong to tracks
data_df[:,0]=data_df[:,0].astype(int)
data_df[:,1]=data_df[:,1].astype(int)

prop_prop = ['t', 'generation', 'root', 'parent', 'minor_axis_length', 'major_axis_length', 'area']

for tProp in prop_prop:

    properties[tProp] = cellDataAll.loc[selVector,tProp]

properties['state'] = [5]*len(properties['t'])

Wall time: 50.9 ms


In [28]:
%%time

prop_prop = ['t', 'generation', 'root', 'parent', 'minor_axis_length', 'major_axis_length', 'area']

for tProp in prop_prop:

    properties[tProp] = cellDataAll.loc[selVector,tProp]

properties['state'] = [5]*len(properties['t'])

Wall time: 10 ms


In [27]:
[5]*10

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [22]:
properties

{'t': array([  0,   1,   2, ..., 360, 360, 360]),
 'state': array([5, 5, 5, ..., 5, 5, 5]),
 'generation': array([0, 0, 0, ..., 0, 0, 0]),
 'root': array([   1,    1,    1, ..., 2766, 2767, 2768]),
 'parent': array([   1,    1,    1, ..., 2766, 2767, 2768]),
 'minor_axis_length': array([24.94230475, 25.81359078, 26.75985249, ..., 18.634833  ,
        19.11165778, 22.10400083]),
 'major_axis_length': array([36.3822781 , 37.74096622, 37.64097934, ..., 31.63611169,
        22.73494965, 23.28040989]),
 'area': array([708., 763., 788., ..., 461., 338., 401.])}

In [23]:
cellDataAll.columns

Index(['label', 'area', 'centroid-0', 'centroid-1', 'orientation',
       'major_axis_length', 'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2',
       'bbox-3', 'image', 'mean_intensity-0_nuc', 'mean_intensity-1_nuc',
       'mean_intensity-2_nuc', 'mean_intensity-3_nuc', 't', 'centroid-0_ring',
       'centroid-1_ring', 'mean_intensity-0_ring', 'mean_intensity-1_ring',
       'mean_intensity-2_ring', 'mean_intensity-3_ring', 'track_id', 'parent',
       'generation', 'root', 'accepted', 'background_01',
       'intensity_01_nuc_corr', 'intensity_01_ring_corr', 'background_02',
       'intensity_02_nuc_corr', 'intensity_02_ring_corr', 'background_03',
       'intensity_03_nuc_corr', 'intensity_03_ring_corr', 'background_04',
       'intensity_04_nuc_corr', 'intensity_04_ring_corr', 'DHB_ratio',
       'cyc_D_over_p21'],
      dtype='object')

In [16]:



# change tracks layer
track_layer.data=data_df
#viewer.layers['tracking'].properties = properties

Wall time: 699 ms


In [60]:
data.shape

(67803, 4)

In [71]:
np.min(cellDataAll.track_id)

1.0

In [79]:
np.min(data_df[:,0])

1.0

In [30]:
np.sum(cellDataAll['track_id'] == data[:,0])

ValueError: ('Lengths must match to compare', (65525,), (67803,))

In [27]:
np.min(data,axis=0)

array([1.        , 0.        , 3.96      , 3.69937657])

In [14]:
%%time

# create tracking data from the full data frame
data = cellDataAll.loc[:,['track_id','t','centroid-0','centroid-1']].to_numpy()
data[:,0]=data[:,0].astype(int)

# change tracks layer
viewer.layers['tracking'].data = data
viewer.layers['tracking'].properties = properties

Wall time: 1min 4s


In [366]:
cellDataAll.columns

Index(['label', 'area', 'centroid-0', 'centroid-1', 'orientation',
       'major_axis_length', 'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2',
       'bbox-3', 'image', 'mean_intensity-0_nuc', 'mean_intensity-1_nuc',
       'mean_intensity-2_nuc', 'mean_intensity-3_nuc', 't', 'centroid-0_ring',
       'centroid-1_ring', 'mean_intensity-0_ring', 'mean_intensity-1_ring',
       'mean_intensity-2_ring', 'mean_intensity-3_ring', 'track_id', 'parent',
       'generation', 'root', 'accepted', 'background_01',
       'intensity_01_nuc_corr', 'intensity_01_ring_corr', 'background_02',
       'intensity_02_nuc_corr', 'intensity_02_ring_corr', 'background_03',
       'intensity_03_nuc_corr', 'intensity_03_ring_corr', 'background_04',
       'intensity_04_nuc_corr', 'intensity_04_ring_corr', 'DHB_ratio',
       'cyc_D_over_p21'],
      dtype='object')

In [218]:
%%time
test = (myLabels[myT:,:,:] == myLabel)

Wall time: 1.03 s


In [215]:
test.shape

(160, 2765, 2765)

In [219]:
myT

201

In [220]:
myLabel

483

Wall time: 259 ms


In [225]:
np.max(temp)

50

In [227]:
data[myInd]

array([  50.        ,   29.        , 1545.57985258, 1746.4041769 ])

In [179]:
cellDataAll.loc[(() & ()),:]

2768.0

In [177]:
myT

201

In [178]:
myLabel

483

## Update labels

In [173]:
@viewer.bind_key('u',overwrite=True)
def update_inProgress(viewer):
    
    global cellDataAll

    # update single object function

    ########################################################
    # modify data frame
    ########################################################

    # get the position in time
    myT = viewer.dims.current_step[0]

    # get my label
    myLabel = layer_inProgress.selected_label

    # calculate features of a new cell and store in the general data frame
    cellDataAll = gen.update_dataFrame(myIm,myIm_signal_list,myLabels,cellDataAll,myT,myLabel)

    # add additional columns
    cellDataAll['DHB_ratio'] = cellDataAll[f'intensity_02_ring_corr']/cellDataAll[f'intensity_02_nuc_corr']
    cellDataAll['cyc_D_over_p21'] = cellDataAll[f'intensity_03_nuc_corr']/cellDataAll[f'intensity_04_nuc_corr']

    ########################################################
    # modify tracking layer
    ########################################################

    # get current status of a tracking layer
    data = viewer.layers['tracking'].data
    properties = viewer.layers['tracking'].properties

    # modify the data for the layer
    data, properties = gen.mod_trackLayer(data,properties,cellDataAll,myT,myLabel)

    # change tracks layer
    viewer.layers['tracking'].data = data
    viewer.layers['tracking'].properties = properties

    ########################################################
    # change viewer status
    ########################################################
    viewer.status = f'Frame {myT} was modified.' 

D:\BARC\napari_tracking_manual\general_functions.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\BARC\napari_tracking_manual\general_functions.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


## Mark track as accepted

In [174]:
# right click for accepting points
# toggles status of a track and visualizes it in the accepted objects layer if changed into accepted

@layer_inProgress.mouse_drag_callbacks.append
def accept_track(layer, event):

    if(event.button == 2):
        
        # look up cursor position
        x = int(viewer.cursor.position[1])
        y = int(viewer.cursor.position[2])
        
        # get data
        inProgressTracks = layer_inProgress.data
        
        # check which cell was clicked
        myTrackNum = inProgressTracks[viewer.dims.current_step[0],x,y]
        
        if myTrackNum > 0:

            # check status
            trackStatus = list(cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'])[0]

            # change status of this track
            cellDataAll.loc[cellDataAll.track_id==myTrackNum,'accepted'] = not(trackStatus)

            # regenerate accepted points
            selData=cellDataAll.loc[cellDataAll.accepted==True,:]
            acceptedPoints = np.array([selData['t'],selData['centroid-0'],selData['centroid-1']]).T 

            # update viewer    
            layer_accepted.data = acceptedPoints

## Stack viewer

In [175]:
def update_stack(viewer_stack,myTrack):
        
    # ask for an update
    stack_track,stack_labels,stack_signal_list = gallery_functions.stack_create_all(myIm,myLabels,myIm_signal_list,data,myTrack,imSize=100)
    
    # display new layers
    my_napari.display_set(viewer_stack,stack_track,stack_labels,stack_signal_list,colors_list,names_list,label_contour=0)
    
def update_graph(viewer_stack,myTrack):
    
    # remove previous graph
    h = viewer_stack.window._dock_widgets['']
    viewer_stack.window.remove_dock_widget(h)
        
    # add new graph
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myTrack) 
    h = viewer_stack.window.add_dock_widget(mpl_widget)

@magicgui(call_button='Show Stack')
def show_stack(viewer: Viewer):
    
    # find current track
    myLabel = viewer.layers['objects'].selected_label
    
    viewer_stack = napari.Viewer()
    update_stack(viewer_stack,myLabel)
    
    # select the right label
    viewer_stack.layers['objects'].selected_label = myLabel
    
    # init graph - there must be a more elegant solution (without this global handle)
    mpl_widget = my_napari.create_graph_widget(track_intensity,to_plot_signals,to_plot_colors,to_plot_names,cellDataAll,myLabel) 
    h = viewer_stack.window.add_dock_widget(mpl_widget)
    graph_list.append(h)
    
    # add an update button
    # in the future - connect it to directly changing the selected layer
    @magicgui(call_button='Update Stack')
    def button_stack(viewer_stack: Viewer):

        myTrack = viewer_stack.layers['objects'].selected_label

        # update stack
        update_stack(viewer_stack,myTrack)

        # update graph
        update_graph(viewer_stack,myTrack)

    viewer_stack.window.add_dock_widget(button_stack,area='bottom')
    
viewer.window.add_dock_widget(show_stack,area='bottom')

In [16]:
def update_gallery(myTrack):
    
    global viewer_gallery
    
    # ask for an update
    gallery_track,gallery_labels,gallery_signal_list = gallery_functions.gallery_create_all(myIm,myLabels,myIm_signal_list,data,myTrack,imSize=100)
    
    # display an update
    try:
        viewer_gallery.close()
    except:
        pass

    viewer_gallery = napari.Viewer()
    
    my_napari.display_set(viewer_gallery,gallery_track,gallery_labels,gallery_signal_list,colors_list,names_list,label_contour=2)

In [132]:
# swapping tracks based on 4 points

@viewer.bind_key('r',overwrite=True)
def swap_points(viewer):

    swapPoints = viewer.layers['swapPoints'].data.astype(int)

    # check that you get a good set of swapPoints
    lenTest = (len(swapPoints) == 4)

    if lenTest:

        framesToSwap = list(set(swapPoints[:,0]))
        framesToSwap.sort()
        frameTest = (framesToSwap[1] == framesToSwap[0]+1)


        # read in in-progress tracks
        inProgressTracks = viewer.layers['inProgressStack'].data

        # check tracks
        myTracks = []
        for myPoint in swapPoints:

            myTracks.append(inProgressTracks[tuple(myPoint)])

        trackTest = (len(set(myTracks)) == 2)

        if (lenTest and frameTest and trackTest):

            # get tracks numbers
            track_1 = list((set(myTracks)))[0] 
            track_2 = list((set(myTracks)))[1]

            frame_change = np.max(framesToSwap)


            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

            # change labels layer
            viewer.layers.pop('inProgressStack')
            viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)


            # change graph
            children_1 = set(data[((properties['parent']==track_1) & (properties['generation']>0)),0])

            for child in children_1:

                graph[int(child)] = track_2

            children_2 = set(data[((properties['parent']==track_2) & (properties['generation']>0)),0])

            for child in children_2:

                graph[int(child)] = track_1


            # change tracking data
            track1_change = ((data[:,0]==track_1) & (data[:,1]>=frame_change))
            track2_change = ((data[:,0]==track_2) & (data[:,1]>=frame_change))

            data[track2_change,0] = track_1
            data[track1_change,0] = track_2

            # change info about the parent 
            track1_change = ((properties['parent']==track_1) & (data[:,1]>=frame_change))
            track2_change = ((properties['parent']==track_2) & (data[:,1]>=frame_change))

            properties['parent'][track1_change] = track_2
            properties['parent'][track2_change] = track_1

            # change info about the root
            root1 = properties['root'][track1_change][0]
            root2 = properties['root'][track2_change][0]

            track1_change = ((properties['root']==root1) & (data[:,1]>=frame_change))
            track2_change = ((properties['root']==root2) & (data[:,1]>=frame_change))

            properties['root'][track1_change] = root2
            properties['root'][track2_change] = root1

            # change info about the generation
            gen1 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_1)]
            gen2 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_2)]

            properties['generation'][track1_change] = properties['generation'][track1_change] + gen2 - gen1
            properties['generation'][track2_change] = properties['generation'][track2_change] + gen1 - gen2



            # update tracking data
            viewer.layers['data'].data = data
            viewer.layers['data'].properties = properties
            viewer.layers['data'].graph = graph

            # clean swap points
            viewer.layers['swapPoints'].data = []

            viewer.status='Tracks have been swapped.'

        else:

            viewer.status='Swap points are incorrect.'
    else:

        viewer.status='Swap points are incorrect.'

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [170]:
importlib.reload(gallery_functions)
importlib.reload(my_napari)

<module 'napari_display_functions' from 'D:\\BARC\\napari_tracking_manual\\napari_display_functions.py'>

In [157]:
myInd=582

In [167]:
np.min([x+int(imSize/2),myIm.shape[1]])

1817

In [110]:
mpl_widget = my_napari.create_graph_widget(track_intensity,signal_intensity_ch,colors_list,names_list,cellDataAll,myTrack)

In [99]:
@magicgui(call_button='Update gallery')
def button_gallery(viewer: Viewer):
    
    myTrack = viewer.layers['objects'].selected_label
    update_gallery(myTrack)
    
    mpl_widget = my_napari.create_graph_widget(track_intensity,signal_intensity_ch,colors_list,names_list,cellDataAll,myTrack) 
    viewer_gallery.window.add_dock_widget(mpl_widget)

viewer.window.add_dock_widget(button_gallery,area='bottom')

In [68]:
@track_layer.bind_key('s',overwrite=True)
def update_inProgress(viewer):

    viewer_small = napari.Viewer()
    myTrack = 1 
    
    # create gallery
    
    viewer_small.add_image(myGallery,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

In [199]:
data[data[:,0]==1092,:]

array([[1092.        ,  318.        , 2162.92535971, 2026.29406475],
       [1092.        ,  319.        , 2162.92535971, 2026.29406475],
       [1092.        ,  320.        , 2139.88372093, 2058.0455814 ]])

In [201]:
data[(data[:,0]==7) & (data[:,1]==318),:]

array([[   7.        ,  318.        , 2172.5382482 , 2015.13455069]])

In [ ]:
# fix update of the small stack to nothing (at the moment it keeps the old cell)

In [ ]:
# move graphs from matplotlib to qt and add them vertical lines
# add numbering of the frames to gallery

In [ ]:
# what to do with ghost objects

In [ ]:
# key binding for linking function

# check if parent have different offspring

# no other offspring
# connect points

# other offspring
# connect
# add to the graph

# what if objects didn;t

In [ ]:
# label image should probably become a float image